In [2]:
from openai import OpenAI  # 주석처리
from dotenv import load_dotenv
import os

from langchain_core.chat_history import InMemoryChatMessageHistory  # 메모리에 대화 기록을 저장하는 클래스
from langchain_core.runnables.history import RunnableWithMessageHistory  # 메시지 기록을 활용해 실행 가능한 래퍼wrapper 클래스
from langchain_openai import ChatOpenAI  # 오픈AI 모델을 사용하는 랭체인 챗봇 클래스
from langchain_core.messages import HumanMessage

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 가져오기



model = ChatOpenAI(model="gpt-4o-mini")

# 세션별 대화 기록을 저장할 딕셔너리
store = {}

# 세션 ID에 따라 대화 기록을 가져오는 함수
def get_session_history(session_id: str):
    # 만약 해당 세션 ID가 store에 없으면, 새로 생성해 추가함
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()  # 메모리에 대화 기록을 저장하는 객체 생성
    return store[session_id]  # 해당 세션의 대화 기록을 반환

# 모델 실행 시 대화 기록을 함께 전달하는 래퍼 객체 생성
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [3]:
config = {"configurable": {"session_id": "abc2"}}  # 세션 ID를 설정하는 config 객체 생성

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 난 이형열이야.")],
    config=config,
)

print(response.content)

안녕하세요, 이형열님! 어떻게 도와드릴까요?


In [4]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

당신의 이름은 이형열입니다. 어떻게 도와드릴까요?


In [5]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

response.content

'죄송하지만, 당신의 이름을 알 수 없습니다. 당신의 이름을 알려주시면 좋겠습니다!'

In [6]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="아까 우리가 무슨 얘기 했지?")],
    config=config,
)

response.content

'우리는 당신의 이름인 이형열에 대해 이야기했어요. 혹시 다른 주제에 대해 이야기하고 싶으신가요? 궁금한 점이나 나누고 싶은 이야기가 있다면 말씀해 주세요!'

In [7]:
config = {"configurable": {"session_id": "abc2"}}
for r in with_message_history.stream(
    [HumanMessage(content = "내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐")],
    config=config,
):
    print(r.content, end="|")

|이|형|열|이라는| 이름|은| 한국|에서| 흔|히| 사용|되는| 이름|이|어서|,| 당신|은| 한국| 사람|일| 가능|성이| 높|다고| 생각|합니다|.| 한국| 문화|에| 대해| 몇| 가지| 말씀|드|리|자|면|:

|1|.| **|음|식|**|:| 한국| 음식|은| 다양|하고| 풍|부|한| 맛|을| 자|랑|합니다|.| 김|치|,| 비|빔|밥|,| 불|고|기|,| 떡|볶|이| 등| 다양한| 전|통| 요|리가| 있습니다|.

|2|.| **|전|통|과| 현대|**|:| 한국|은| 전|통|과| 현대|가| 잘| 어|우|러|져| 있는| 나라|입니다|.| 고|대| 궁|궐|과| 현대|식| skyscr|aper|가| 공|존|하며|,| K|-|POP|과| 전|통| 음악|,| 춤|이| 함께| 즐|길| 수| 있습니다|.

|3|.| **|명|절|**|:| 설|날|과| 추|석|은| 한국|의| 대표|적인| 명|절|로|,| 가족|이| 모|여| 함께| 시간을| 보내|고| 전|통| 음|식을| 나|누|는| 중요한| 날|입니다|.

|4|.| **|한|글|**|:| 한국|의| 공식| 문자| 체|계|인| 한|글|은| 과|학|적인| 설|계|로| 인해| 배우|기| 쉽|고|,| 모든| 사람|에게| 알|기| 쉽게| 되어| 있습니다|.

|5|.| **|예|절|**|:| 한국| 문화|에서| 예|의|는| 매우| 중요|하게| 여|겨|집|니다|.| 나|이에| 따라| 존|댓|말|을| 사용|하고|,| 인|사|나| 자|리를| 양|보|하는| 등의| 행동|이| 중|시|됩니다|.

|이|와| 관련|해| 더| 궁|금|한| 점|이나| 다른| 문화|에| 대해| 알고| 싶|으|신| 것이| 있다|면| 말씀|해| 주세요|!||